# 02 - Data Cleaning

## Objective
Load the latest raw snapshots for each ticker, clean and standardise the structure, then save a single tidy dataset for analysis and modelling.

## Inputs
- Raw CSV snapshots in `data/raw/<version>/`
- Version label (e.g. v1)

## Outputs
- Cleaned dataset saved to `data/processed/<version>/clean_prices_<version>_<timestamp>.csv`
- Basic data quality checks (shape, missing values)

## CRISP-DM Stage
Data Preparation

In [1]:
# Make the project root importable (so `import src...` works in notebooks)
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()  # notebooks live in jupyter_notebooks/
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("Project root added to sys.path:", PROJECT_ROOT)

In [2]:
from pathlib import Path

import pandas as pd

from src.config import DEFAULT_TICKERS, DEFAULT_VERSION, get_paths

In [ ]:
VERSION = DEFAULT_VERSION
TICKERS = DEFAULT_TICKERS

paths = get_paths(VERSION)
RAW_DIR = paths.raw_dir
PROCESSED_DIR = paths.processed_dir
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print("Raw dir:", RAW_DIR)
print("Processed dir:", PROCESSED_DIR)
print("Tickers:", ", ".join(TICKERS))

In [ ]:
from typing import Dict


def latest_snapshot_for_ticker(raw_dir: Path, ticker: str, version: str) -> Path:
    pattern = f"{ticker}_raw_{version}_*.csv"
    matches = sorted(raw_dir.glob(pattern))
    if not matches:
        raise FileNotFoundError(f"No raw snapshots found for {ticker} in {raw_dir}")
    return matches[-1]


files: Dict[str, Path] = {
    t: latest_snapshot_for_ticker(RAW_DIR, t, VERSION) for t in TICKERS
}

for t, f in files.items():
    print(t, "->", f.name)

In [ ]:
REQUIRED_COLUMNS = ["Date", "Open", "High", "Low", "Close", "Adj_Close", "Volume", "Ticker"]


def clean_prices(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Standardise column names
    df.columns = [str(c).strip().replace(" ", "_") for c in df.columns]

    # Required columns checks
    if "Ticker" not in df.columns:
        raise KeyError("Expected 'Ticker' column is missing.")
    if "Date" not in df.columns:
        raise KeyError("Expected 'Date' column is missing.")

    # Ensure Adj_Close exists (fallback to Close)
    if "Adj_Close" not in df.columns and "Close" in df.columns:
        df["Adj_Close"] = df["Close"]

    # Parse dates
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"])

    # Coerce numerics
    for col in ["Open", "High", "Low", "Close", "Adj_Close", "Volume"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # Drop duplicates + sort
    df = df.drop_duplicates(subset=["Ticker", "Date"])
    df = df.sort_values(["Ticker", "Date"])

    # Ensure the output schema is consistent (prevents future concat bugs)
    for col in REQUIRED_COLUMNS:
        if col not in df.columns:
            df[col] = pd.NA

    df = df[REQUIRED_COLUMNS]

    return df

In [ ]:
frames = []

for ticker, path in files.items():
    temp = pd.read_csv(path)
    temp = clean_prices(temp)
    frames.append(temp)

clean_df = pd.concat(frames, ignore_index=True)

# Quick preview (3 rows per ticker so it never looks like NaNs)
display(clean_df.groupby("Ticker").head(3))